## 1. 디렉토리 변경

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/recipekogpt2

/content/drive/MyDrive/recipekogpt2


## 2. 데이터 불러오기, 데이터 확인

데이터는 딕셔너리 형태로 저장되어 있으며, 재료부분은 값이 리스트 형태입니다.


data_dir에 recipe_preprocessed.pickle파일이 있는 디렉토리 주소를 넣어주세요

In [5]:
import pickle
data_dir = '/content/drive/MyDrive/processed_data'
with open(f'{data_dir}/recipe_preprocessed.pickle', "rb") as fr:
    dataframe = pickle.load(fr)
dataframe

,음식명,재료,레시피,ingredient_preprocessed
0,칼륨 듬뿍 고구마죽,"고구마죽\n고구마 100g(2/3개), 설탕 2g(1/3작은술), 찹쌀가루 3g(2...",고구마는 깨끗이 씻어서 껍질을 벗기고 4 정도로 잘라준다. 찜기에 고구마를 넣고 2...,"[고구마, 설탕, 찹쌀가루, 물, 잣]"
1,누룽지 두부 계란죽,"채소준비\n애호박 30g(1/6개), 표고버섯 20g(2개), 당근 5g(3×2×1...",깨끗이 씻어 손질한 애호박 당근과 기둥을 뗀 표고버섯을 잘게 다지듯이 썬다. 누룽지...,"[애호박, 표고버섯, 당근, 누룽지, 순두부, 달걀, 참기름, 소금, 참깨, 후추]"
2,오색지라시 스시,"초밥\n밥 210g(1공기)\n배합초\n식초 20g(1⅓큰술), 설탕 10g(2작은...",배합초는 중불에서 설탕이 녹을 때까지 저어가면서 녹인다. 뜨거운 밥에 배합초를 넣어...,"[초밥, 밥, 배합초, 식초, 설탕, 소금, 오색, 지라시, 달걀노른자, 표고버섯,..."
3,두부 곤약 나물 비빔밥,"두부 곤약잡곡밥\n두부 110g(⅓모), 흰쌀 15g, 현미쌀 3g, 찹쌀 3g, ...",고사리는 상태에 따라 2-5시간 정도 불린 후 30분 정도 삶아 찬물에 헹구어 물기...,"[두부, 곤약, 잡곡밥, 흰쌀, 현미쌀, 찹쌀, 실곤약, 나물, 콩나물, 표고버섯,..."
4,저염 간장을 이용한 닭개장 비빔밥,"흑미밥\n쌀 90g, 검은 쌀 10g\n닭가슴살 삶기\n닭가슴살 40g(1/3개),...",쌀은 30분 정도 불린 후 물을 1 하여 밥을 짓는다. 숙주는 끓는 소금물에 넣어 ...,"[흑미밥, 쌀, 검은, 닭가슴살, 월계수, 잎, 후추, 나물, 숙주, 토란대, 고사..."
...,...,...,...,...
22769,얼큰 콩나물 수제비,"콩나물 50g, 황태머리 40g, 고추장 10g, 밀가루 100g, 고춧가루 5g,...",밀가루에 소금을 섞어 반죽한 뒤 숙성시킨다. 황태머리를 구운 뒤 찬물에 넣고 끓여 ...,"[콩나물, 황태머리, 고추장, 밀가루, 고춧가루, 소금]"
22770,"단호박 생선탕수, 키위소스","동태살 120g, 단호박 10g, 당근 15g, 오이 15g, 양파 15g, 전분 ...",따뜻한 물에 부린 목이 버섯 동태 살 당근 오이 양파를 먹기 좋은 크기로 썬다. 동...,"[동태살, 단호박, 당근, 오이, 양파, 전분, 계란, 튀김가루, 목이버섯, 식초,..."
22771,낙지모듬초말이,"낙지 30g, 가지 12g, 오이 7g, 표고버섯 24g, 당근 10g, 미나리 1...",낙지는 살짝 데 치고 한 입 크기로 썬다. 오이와 당근 표고 가지는 얇게 동 그란 ...,"[낙지, 가지, 오이, 표고버섯, 당근, 미나리, 홍고추, 설탕, 식초, 물, 사과..."
22772,삼각김밥 빵,"(반죽재료) 쌀가루 200g, 우유 80g, 물 15ml, 르뱅 발효종 160g, ...",속 재료 양파 마늘 토마토 파프리카 닭 가슴살을 다져 올리브유를 두른 팬에 볶아 준...,"[쌀가루, 우유, 물, 르뱅, 발효종, 달걀, 소금, 올리브유, 양파, 당근, 먹물..."


In [6]:
import pandas as pd
dataframe.rename(columns ={'음식명':'title','재료':'_','레시피':'instructions', 'ingredient_preprocessed' : 'ingredients'}, inplace = True)
df_t = dataframe.loc[:,['title','ingredients','instructions']]
df_t.head()

,title,ingredients,instructions
0,칼륨 듬뿍 고구마죽,"[고구마, 설탕, 찹쌀가루, 물, 잣]",고구마는 깨끗이 씻어서 껍질을 벗기고 4 정도로 잘라준다. 찜기에 고구마를 넣고 2...
1,누룽지 두부 계란죽,"[애호박, 표고버섯, 당근, 누룽지, 순두부, 달걀, 참기름, 소금, 참깨, 후추]",깨끗이 씻어 손질한 애호박 당근과 기둥을 뗀 표고버섯을 잘게 다지듯이 썬다. 누룽지...
2,오색지라시 스시,"[초밥, 밥, 배합초, 식초, 설탕, 소금, 오색, 지라시, 달걀노른자, 표고버섯,...",배합초는 중불에서 설탕이 녹을 때까지 저어가면서 녹인다. 뜨거운 밥에 배합초를 넣어...
3,두부 곤약 나물 비빔밥,"[두부, 곤약, 잡곡밥, 흰쌀, 현미쌀, 찹쌀, 실곤약, 나물, 콩나물, 표고버섯,...",고사리는 상태에 따라 2-5시간 정도 불린 후 30분 정도 삶아 찬물에 헹구어 물기...
4,저염 간장을 이용한 닭개장 비빔밥,"[흑미밥, 쌀, 검은, 닭가슴살, 월계수, 잎, 후추, 나물, 숙주, 토란대, 고사...",쌀은 30분 정도 불린 후 물을 1 하여 밥을 짓는다. 숙주는 끓는 소금물에 넣어 ...


In [7]:
df_t.shape

(21727, 3)

null값 확인

In [8]:
df_t.isnull().any()

title           False
ingredients     False
instructions    False
dtype: bool

In [9]:
df_drop = df_t.dropna(axis=0)

데이터프레임 형태를 모델에 알맞게 딕셔너리 형태로 변형합니다.

In [10]:
data = df_drop.to_dict('records') 
data[0:3]

[{'ingredients': ['고구마', '설탕', '찹쌀가루', '물', '잣'],
  'instructions': '고구마는 깨끗이 씻어서 껍질을 벗기고 4 정도로 잘라준다. 찜기에 고구마를 넣고 20-30분 정도 삶아 주고 블렌더나 체를 이용하여 잘 으 깨어 곱게 만든다. 고구마와 물을 섞어 끓이면서 찹쌀가루로 농도를 맞추고 설탕을 넣어 맛을 낸다. 잣을 팬에 노릇하게 볶아 다져서 고구마 죽에 섞는다. 기호에 따라 고구마를 튀겨 얹어 먹어도 좋다.',
  'title': '칼륨 듬뿍 고구마죽'},
 {'ingredients': ['애호박',
   '표고버섯',
   '당근',
   '누룽지',
   '순두부',
   '달걀',
   '참기름',
   '소금',
   '참깨',
   '후추'],
  'instructions': '깨끗이 씻어 손질한 애호박 당근과 기둥을 뗀 표고버섯을 잘게 다지듯이 썬다. 누룽지는 1 정도로 잘게 부숴준다. 냄비에 참기름을 두르고 썰어 놓은 채소를 볶다가 누룽지와 물을 넣고 끓인다. 누룽지가 살짝 퍼지면 순두부를 넣고 흰 후추와 소금을 넣는다. 죽이 끓으면 달걀을 풀어 넣고 한 소끔 끓여낸 후 참깨를 뿌려 마무리한다.',
  'title': '누룽지 두부 계란죽'},
 {'ingredients': ['초밥',
   '밥',
   '배합초',
   '식초',
   '설탕',
   '소금',
   '오색',
   '지라시',
   '달걀노른자',
   '표고버섯',
   '새송이버섯',
   '새우',
   '홍피망',
   '청피망',
   '양파',
   '대두유',
   '참기름',
   '식용유'],
  'instructions': '배합초는 중불에서 설탕이 녹을 때까지 저어가면서 녹인다. 뜨거운 밥에 배합초를 넣어서 밥알이 으깨지지 않게 고루 저어가면서 밥을 체온 정도로 식힌다. 표고버섯은 기둥을 떼고 끓는 물에 데친 후 찬물에 헹궈 물기를 없애고 얇게 썰어 달궈진 팬에 참기름을 두르고 볶는다. 새송이버섯은 세로로 

## 3. 데이터 구조 변경

딕셔너리 형태의 데이터를 txt형태로 바꿔줍니다. 이 때, 각 부분(ingredients, title, instructions)의 시작과 끝 부분에 토큰을 붙여줍니다. 토큰은 추후 GPT-2가 훈련할 때 각 부분이 시작하고 끝난다는 것을 나타내줍니다. 또한 각 재료 사이에는 '$'문자를 삽입해 재료를 구분해줍니다.

In [11]:
from preprocessing.dependency import parent_dir
from preprocessing.common.basics import *
from preprocessing.common.save import make_dir, save_pickle, load_pickle, save
import random
import tqdm

In [12]:

def txt(v, fields, mode = 'train'):
    """ transform the recipe dict to string
    
        Args:
          v: A dict of recipe information
          fields: an order list, the last is the field to predict
          mode: string

        Returns:
          In test mode, return string X, y
          In train mode, return string X + y
    """
    to_write = ''
    for field in fields:
        if field == 'title':
            name = v['title']
            to_write += '<unused0>'+name+'<unused1>'
        if field == 'ingredients':
            ingredients = v['ingredients']
            to_write += '<unused2>'+'$'.join(ingredients)+'$<unused3>'
        if field == 'instructions':
            instructions = v['instructions']
            to_write += '<unused4>'+ instructions +'<unused5>'
    to_write += '\n'
            
    if mode == 'train':
        return to_write
                                                     
    elif mode == 'test':
        field_to_predict = '<start-%s>'%fields[-1]
        to_X, to_y = to_write.split(field_to_predict)
        return to_X+field_to_predict, to_y

class to_gpt2:
    def __init__(self, data, ls = None):
        if not ls:
            ls = list(range(len(data)))
        random.seed(42)
        random.shuffle(ls)
        #self.ls_test = ls[:2173] #10, 10, 80, test, val, train
        self.ls_val = ls[:2173] #수정, test set 없앰.
        self.ls_train = ls[2173:]
        self.data = data
        
    def test(self, ls, filename, overwrite = False, is_val = False):
        to_write = ''
        # order of field and the inital of the field to predict
        orders = [(['title','ingredients','instructions'], 'd'),
                  (['title','instructions','ingredients'], 'i'),
                  (['ingredients','instructions','title'], 't')]
        
        for i, v in tqdm.tqdm(enumerate(self.data)):
            if i in ls:
                for order, init in orders:
                    X, y = txt(v, order, mode = 'test')
                    self.save(filename+'X/%d'%(i)+'%s.txt'%(init), X, overwrite)
                    self.save(filename+'y/%d'%(i)+'%s.txt'%(init), y, overwrite)
        
    def save(self, filename, to_write, overwrite = False):
        make_dir(filename)
        if os.path.isfile(filename) == True and overwrite == False:
            print('already exists'+filename)
        else:    
            with open(filename,'w') as f:
                f.write('%s' % to_write)
                
    def make_txt(self, ls, filename, tag, overwrite = False):
        make_dir(filename)
        f = open(f'{filename}/{tag}', 'w')
        for i, v in tqdm.tqdm(enumerate(self.data)):
            if i in ls:
                f.writelines(self.encode_recipe(self.data[i]))      
        print(f'{filename}/{tag}')
        f.close()
        
    def encode_recipe(self,recipe):
        return txt(recipe, ['title','ingredients','instructions'])
    
    def fast_txt(self, filename, overwrite = False):
        self.make_txt(self.ls_train, filename, tag = 'recipegpt_train.txt')
        self.make_txt(self.ls_val, filename, tag = 'recipegpt_val.txt')
        #self.make_txt(self.ls_test, filename, tag = 'recipegpt_test.txt')

클래스 선헌 후, 인스턴스를 만듭니다. 처리된 데이터를 저장할 디렉토리를 선언합니다.

In [13]:
model = to_gpt2(data)
filename = './data/recipegpt_l300_processed'

In [14]:
model.fast_txt(filename)

21727it [00:08, 2635.01it/s]


./data/recipegpt_l300_processed/recipegpt_train.txt


21727it [00:01, 17431.49it/s]

./data/recipegpt_l300_processed/recipegpt_val.txt
